# Modeling the Hourly Rate

This code trains a model to predict what a prospective freelancers hourly rate should be. The model used is a random forest regression model. I also built an AdaBoosted Decision Tree model, but it performs much worse (using 5-fold cross-validation) compared with the turned random forest regression model.

I've also built a k-modes (cluster with categorical/numeric mixed data) model. It is currently being tested.

## Importing Libraries and Data

In [1]:
# Data Analysis and Modeling
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

# Random Forest Regression
from sklearn.ensemble import RandomForestRegressor

# AdaBoost Decision Tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

# Cross-Validation and Accuracy Measures
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

# Model Serializing (export to web-app)
import pickle

# Packages for PostgreSQL Import
import psycopg2
import os

# Packages for K-Modes Cluster
from kmodes.kmodes import KModes

## Importing Data from SQL

In [2]:
# Names for accessing SQL database
# Ideally move these to environment variables.

dbname = "freelance_db"
username = os.environ['USER']
pswd = os.environ['SQLPSWD']

In [ ]:
# Connect to SQL Database
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

# Import Analysis Dataset
sql_query = """SELECT * FROM analysis_table;"""
analysis_dt = pd.read_sql_query(sql_query, con)

sql_query = """SELECT * FROM embeddings_table;"""
embeddings = pd.read_sql_query(sql_query, con)

In [ ]:
# Doing some cleaning. Removing columns that have a continuous analog in the model.
analysis_dt = analysis_dt.drop(['has_rating'], axis=1)
analysis_dt = analysis_dt.drop(['rating'], axis = 1)
analysis_dt = analysis_dt.drop(['index'], axis=1)
analysis_dt = analysis_dt.drop(['months_active'], axis = 1)

In [ ]:
analysis_dt.info

# Modeling
## Estimating a Random Forest Regression to Estimate Hourly-Rate

Notes:
    - Remove extreme outliers
    - Next model probability of getting a job?
    - Convert to a clustering algorithm and present a range of estimates?
    - Bootstrap to get a range of estimates?
    - Make suggestions (Increase bio length by 10 and you can charge X more)
        - "Here are bios for users similar to you" etc. etc.
    - Focus on helping users build up their profiles

In [ ]:
# Converting data to numpy arrays and saving column names
y = np.array(analysis_dt['hourly_rate'])

analysis_dt = analysis_dt.drop(['hourly_rate'], axis = 1)
feature_list = list(analysis_dt.columns)

X = np.array(analysis_dt)

In [ ]:
# Creating train and test splits
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
# Getting Baseline
baseline_preds = y_train.mean()

# Baseline Errors
baseline_errors = abs(baseline_preds - y_train)

# Baseline:
print('Mean Absolute Error (naive average): ', round(np.mean(baseline_errors), 2))

In [ ]:
# References for GridSearchCV
# 1 - https://scikit-learn.org/stable/modules/ensemble.html#random-forest-parameters
# 2 - https://stackoverflow.com/questions/35097003/cross-validation-decision-trees-in-sklearn
# References for Random Forest Regression
# 1 - https://towardsdatascience.com/random-forest-in-python-24d0893d51c0
# 2 - https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

# Creating parameter grid
parameters = {"max_features": ["auto", "sqrt"], "n_estimators": [
    100, 300, 500, 700, 900, 1100, 1300, 1500, 1700, 2000]}

# Creating Grid Search Class
clf = GridSearchCV(RandomForestRegressor(), parameters, n_jobs=4, verbose=True, 
                   scoring="neg_mean_squared_error",
                   refit=True)

# Running Grid Search
clf.fit(X=X_train, y=y_train)

# Saving best model
rfr_model = clf.best_estimator_

# Printing results
print(round(clf.best_score_,2), clf.best_params_)

In [ ]:
# Plotting Grid Search CV Results
# Saving results to pandas data frame
grid_search_results = pd.DataFrame(clf.cv_results_)

# Plotting
sns.scatterplot(x=list(grid_search_results.index), 
               y=grid_search_results.mean_test_score*-1)

### Evaluating Model Performance

In [ ]:
# Loading serialized model from pickle
# filename = os.environ['PWD'] + "/scripts/finalized_model.sav"
# rfr_model = pickle.load(open(filename, 'rb')) 

In [ ]:
# Predict
preds = rfr_model.predict(X_train)

In [ ]:
# Calculate absolute error (comparison with baseline)
errors = abs(preds - y_train)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_train, preds))

# The coefficient of determination (R2)
print('Coefficient of determination: %.2f'
      % r2_score(y_train, preds))

# Does much better than the mean with regard to mean absolute error
print("A ", round((np.mean(errors)-19.49)/19.49,3)*-100,
      "percent improvement over the mean")

In [ ]:
# Plotting True Hourly Rates vs Predicted Hourly Rates
sns.scatterplot(x = y_train, y = preds)

## Fitting AdaBoost Model
The reason to fit this model is that iteratively re-weights observations based on how poorly the model predicts them. This pushes the model to finding a way to predict observations other models may miss. I am concerned, however, if it will overfit to outliers.

It performs poorly compared to the random forest regression.

In [ ]:
# Creating parameter grid
parameters = {"n_estimators": [1, 3, 5, 10, 25, 50, 75, 100, 125, 150],
              "base_estimator__max_depth": [3, 5, 7, 9, 10, 11]}

regr_1 = DecisionTreeRegressor()
regr_2 = AdaBoostRegressor(base_estimator=regr_1)

# Creating Grid Search Class
clf = GridSearchCV(regr_2, parameters, n_jobs=4, verbose=True)

# Running Grid Search
clf.fit(X=X_train, y=y_train)

# Saving best model
adaboost_model = clf.best_estimator_

# Printing results
print(round(clf.best_score_, 2), clf.best_params_)

### Evaluating Model Performance

In [ ]:
# Predict
preds_ada = adaboost_model.predict(X_train)

In [ ]:
# Calculate absolute error (comparison with baseline)
errors = abs(preds_ada - y_train)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# The mean squared error 
print('Mean squared error: %.2f'
      % mean_squared_error(y_train, preds_ada))

# The coefficient of determination (R2)
print('Coefficient of determination: %.2f'
      % r2_score(y_train, preds_ada))

# Does much better than the mean with regard to mean absolute error
print("A ", round((np.mean(errors)-19.12)/19.12,2)*-100,
      "percent improvement over the mean")

In [ ]:
# Plotting True Hourly Rates vs Predicted Hourly Rates
sns.scatterplot(x = y_train, y = preds)

## Exporting Model

Putting the random forest regression model (tuned) into production.

In [ ]:
# Saving model with pickle
filename = '/Users/Metaverse/Desktop/Insight/projects/myrate/scripts/finalized_model.sav'
pickle.dump(rfr_model, open(filename, 'wb'))

# Model Testing Ground

## Model with embeddings

I can change the number of embeddings in the bios_word2vec notebook.

In [5]:
# Removing index col from embeddings
embeddings = embeddings.drop(['index'], axis = 1)

# Merging on profile_url
test = analysis_dt.merge(embeddings, on = "profile_url")
test.shape

KeyError: 'profile_url'

## Building K-Modes Model to Cluster Users
This is an experimental section. I want to try and create clusters to then provide a range of values ones hourly rate may take. However, the problem with clustering methods (as always) is that it is more of an art than actual science for determining when to stop. I don't think there are very clear clusters in this data. I may be better off running word2vec on the bios (large text) and clustering off that.

In [ ]:
# Found K-Modes on stackoverflow
# Following the package documentation
# Ref: https://pypi.org/project/kmodes/
# Ref: https://stackoverflow.com/questions/42639824/python-k-modes-explanation
# Ref: https://www.kaggle.com/ashydv/bank-customer-clustering-k-modes-clustering

In [ ]:
# Modeling with K Modes
cost = []
for num_clusters in list(range(1, 21)):
    kmode = KModes(n_clusters=num_clusters, init="Huang", verbose=0, random_state=42)
    kmode.fit_predict(X)
    cost.append(kmode.cost_)
    print("Finished Cluster: " + str(num_clusters))

In [ ]:
# Plotting cost function result vs number of clusters
number_of_clusters = np.array([i for i in range(1, 21, 1)])
plt.plot(number_of_clusters, cost)

In [ ]:
kmode = KModes(n_clusters=6, init='Huang', verbose=0)
clusters = kmode.fit_predict(X)

kmodes = kmode.cluster_centroids_
shape = kmodes.shape

for i in range(shape[0]):
    if sum(kmodes[i, :]) == 0:
        print("\ncluster " + str(i) + ": ")
        print("no-skills cluster")
    else:
        print("\ncluster " + str(i) + ": ")
        cent = kmodes[i, :]
        for j in analysis_dt.columns[np.nonzero(cent)]:
            print(j)

In [ ]:
clust_assigned = kmode.predict(X)

In [ ]:
unique, counts = np.unique(clust_assigned, return_counts=True)
dict(zip(unique, counts))

In [ ]:
analysis_dt['cluster'] = list(clust_assigned)

In [ ]:
analysis_dt[analysis_dt.cluster == 0]